In [1]:
import torch
from torch import onnx
from torchvision import models

model = models.vgg16(num_classes=2)
model.load_state_dict(torch.load("../models/VGG16.pt", map_location=torch.device('cpu')))

model.eval()

dummy_input = torch.randn(1, 3, 224, 224)
onnx.export(model=model, args=dummy_input, f="../models/VGG16.onnx")

In [3]:
import time
import torch
import onnxruntime as ort
from PIL import Image
from torchvision import models
from torchvision import transforms

def to_numpy(tensor):
    return tensor.detach().cpu().numpy()

image = Image.open("./datasets/images/cat.jpg")

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48235, 0.45882, 0.40784], std=[1.0/255.0, 1.0/255.0, 1.0/255.0])
])

input = transform(image).unsqueeze(0)
model = models.vgg16(num_classes=2)
model.load_state_dict(torch.load("../models/VGG16.pt", map_location=torch.device('cpu')))
model.eval()

with torch.no_grad():
    start_time = time.time()
    output = model(input)
    end_time = time.time()

    print("PyTorch inference time:", output, end_time - start_time)

ort_session = ort.InferenceSession("../models/VGG16.onnx")

start_time = time.time()
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input)}
ort_outs = ort_session.run(output_names=None, input_feed=ort_inputs)
end_time = time.time()
print("ONNX inference time:", ort_outs, end_time - start_time)

PyTorch inference time: tensor([[ 25.6561, -22.3740]]) 0.09152388572692871
ONNX inference time: [array([[ 25.656073, -22.373951]], dtype=float32)] 0.14581513404846191
